In [8]:
import os
import random
import gym
import tensorflow as tf
import numpy as np
import imageio
from skimage.transform import resize

# A Journey through the Evolution of Deep Learning and the Quest for Human-Level Control via Deep Reinforcement Learning

### By Atanas Vasev

# 1. Introduction

Despite the seemingly futuristic and complex nature of the term 'Deep Learning,' its roots can be traced back to 1940 with the pioneering work of Warren McCulloch and Walter Pitts. They laid the foundation for neural networks with their influential paper titled 'A Logical Calculus of Ideas Immanent in Nervous Activity.' However, progress in neural networks faced challenges for decades due to limitations in computing power and data availability. It wasn't until the early 21st century that significant advancements reignited interest, with a transformative moment occurring in 2012 during the ImageNet Large Scale Visual Recognition Challenge.

This pivotal moment was characterized by the breakthrough achieved by a team led by Alex Krizhevsky, Ilya Sutskever, and Geoffrey Hinton. They demonstrated a groundbreaking leap in image classification accuracy using a deep neural network architecture known as AlexNet. This success marked the resurgence of deep learning, showcasing its superior capabilities in handling complex tasks.

The success of AlexNet opened the door for extensive exploration and development in deep neural networks. Researchers delved into designing more sophisticated architectures, introducing techniques like convolutional layers for image processing. This momentum catalyzed breakthroughs in various domains, including natural language processing and speech recognition.

The landscape of deep learning further evolved with the introduction of reinforcement learning techniques, notably highlighted by DeepMind's Deep Q-Learning algorithm. In particular, Deep Q-Learning garnered attention with its remarkable performance in the game of Breakout. DeepMind demonstrated the ability of the algorithm to achieve superhuman performance by learning optimal strategies through reinforcement learning.

As researchers progressed, they continued to refine and expand reinforcement learning methodologies. The focus on enhancing training techniques and algorithmic advancements led to further breakthroughs, making reinforcement learning a key player in the evolution of deep learning.

The dynamic journey of deep learning, from the foundational work of McCulloch and Pitts to the complex, versatile neural networks of today, is exemplified by the continuous exploration of novel architectures, training methodologies, and applications. In the realm of reinforcement learning, the impact of algorithms like Deep Q-Learning serves as a testament to the ongoing commitment to unraveling the full potential of deep learning in replicating and surpassing human-level control, particularly in challenging domains such as Breakout.

In this paper, I will attempt to replicate DeepMind's Deep Q-Learning algorithm. The primary objective is to achieve a greater understanding not only of reinforcement learning but also of the broader topic of deep learning. The reason I chose this particular paper and topic is that this type of deep learning architecture can serve as a valuable case study for its success in achieving human-level performance in complex tasks. Deep Q-Learning, proposed by researchers at DeepMind, has demonstrated remarkable capabilities in mastering complex environments through reinforcement learning. By replicating and dissecting the nuances of this algorithm, I aim to unravel insights into the principles of reinforcement learning and the intricate mechanisms that contribute to the success of deep learning models.

The journey involves implementing, testing, and fine-tuning the Deep Q-Learning algorithm, aiming not only for successful replication but also for a deeper comprehension of the underlying principles. Furthermore, this endeavor provides an opportunity to explore the broader implications of deep learning, such as its adaptability to different domains and its potential for scalable and efficient learning.

As the paper progresses, I will delve into the historical context of deep learning, tracing its evolution from its early theoretical foundations to the recent breakthroughs in algorithmic advancements. By synthesizing this historical narrative with practical implementation, the aim is to provide a comprehensive exploration of deep learning, emphasizing its transformative impact on artificial intelligence.

Through the lens of replicating Deep Q-Learning, this paper seeks to contribute to the ongoing discourse on the challenges, advancements, and future prospects of deep learning. By gaining insights into the inner workings of this algorithm, we can further our understanding of the broader implications of reinforcement learning and its potential applications in real-world scenarios.

In 2013 a London based startup called DeepMind published a groundbreaking paper called ["Playing Atari with Deep Reinforcement Learning"](https://arxiv.org/abs/1312.5602) on arXiv: The authors presented a variant of Reinforcement Learning called Deep Q-Learning that is able to successfully learn control policies for different Atari 2600 games receiving only screen pixels as input and a reward when the game score changes. The agent even surpasses human expert players in some of those games! This is an astonishing result because previously “AIs” used to be limited to one single game, for instance, chess, whereas in this case the types and contents of the games in the Arcade Learning Environment vary significantly and yet no adjustment of the architecture, learning algorithm or hyperparameters is needed.

No wonder DeepMind was bought by Google for 500 Million Dollars. The company has since been one of the leading institutions advancing Deep Learning research and a later article discussing DQN(Deep Q-Networks) has been published in Nature

# 2. Deep Q-Learning

Before we start the implementation we need to cover some important concepts in the field of reinforcement learning: Q-learning and deep Q-learning.

### 2.1 Reinforcement Learning
A key differentiator of reinforcement learning from supervised or unsupervised learning is the presence of two things:

 * An environment: This could be something like a maze, a video game, the stock market, and so on.
 * An agent: This is the AI that learns how to operate and succeed in a given environment

The way the agent learns how to operate in the environment is through an iterative feedback loop. The agent first takes an action and as a result, the state will change based on the rewards either won or lost.

**The goal of the agent is to maximize its cumulative expected reward.**

### 2.2 Bellman Equation
The [Bellman equation](https://en.wikipedia.org/wiki/Bellman_equation?ref=mlq.ai) is a fundamental concept in reinforcement learning.

The concepts we need to understand the Bellman Equation include:

$s$ - State

$a$ - Action

$R$ - Reward

$γ$ - Discount factor

The Bellman Equation was introduced by Dr. Richard Bellman (who's known as the Father of dynamic programming) in 1954 in the paper: [The Theory of Dynamic Programming](https://www.rand.org/content/dam/rand/pubs/papers/2008/P550.pdf?ref=mlq.ai).

To understand this, let's look at an example from freeCodeCamp, in which we ask the question:

**How do we train a robot to reach the end goal with the shortest path without stepping on a mine?**

<img src="./IMG/Robot_Maze.png" width="300">

Below is the reward system for this environment:

 - 1 point at each step. This is to encourage the agent to reach the goal in the shortest path.
   
 - 100 for stepping on a mind and the game ends
   
 + 1 for landing on a⚡️
 + 
 + 100 for reaching the End

To solve this we introduce the concept of a Q-table.

A Q-table is a lookup table that calculates the expected future rewards for each action in each state. This lets the agent choose the best action in each state.

In this example, our agent has 4 actions (up, down, left, right) and 5 possible states (Start, Blank, Lightning, Mine, End).

So the question is: how do we calculate the maximum expected reward in each state, or the values of the Q-table?

We learn the value of the Q-table through an iterative process using the Q-learning algorithm, which uses the Bellman Equation.

Here is the Bellman equation for deterministic environments:

$V(s)=max_aR(s,a)+γV(s′))$

Here's a summary of the equation from our earlier Guide to Reinforcement Learning:

* The value of a given state is equal to max action, which means of all the available actions in the state we're in, we pick the one that maximizes value.

* We take the reward of the optimal action $a$ in state $s$ and add a multiplier of $γ$, which is the discount factor that diminishes our reward over time.

* Each time we take an action we get back the next state: $s'$. This is where dynamic programming comes in, since it is recursive we take $s$ and put it back into the value function $V(s)$.

### 2.3 Markov Decision Processes (MDPs)

Now that we've discussed the concept of a Q-table, let's move on to the next key concept in reinforcement learning: Markov decision processes, or MDPs.

First let's review the difference between deterministic and non-deterministic search.

Deterministic Search:
 * Deterministic search means that if the agent tries to go up (in our maze example), then with 100% probability it will in fact go up.

Non-Deterministic Search:
*  Non-deterministic search means that if our agent wants to up, there could be an 80% chance it goes up, a 10% it goes left, and 10% it goes right, for example. So there is an element of randomness in the environment that we need to account for this.

This is where two new concepts come in: Markov processes and Markov decision processes.

Here's the definition of [Markov process](https://en.wikipedia.org/wiki/Markov_property?ref=mlq.ai) from Wikipedia:

 **"A stochastic process has the Markov property if the conditional probability distribution of future states of the process (conditional on both past and present states) depends only upon the present state, not on the sequence of events that preceded it. A process with this property is called a Markov process."**

To simplify this, in an environment with a Markov property the way the environment is designed in such a way that what happens in the future doesn't depend on the past.

Now here's the definition of a [Markov decision process](https://en.wikipedia.org/wiki/Markov_decision_process?ref=mlq.ai):

**"A Markov decision process provides a mathematical framework for modeling decision making in situations where outcomes are partly random and partly under the control of a decision maker."**

A Markov decision process is the framework that the agent will use in order to operate in this partly random environment.

Let's now build on our Bellman equation, which as a reminder is:

$V(s)=max_aR(s,a)+γV(s′))$

Now that we have some randomness we don't actually know what $s′$ we'll end up in. Instead, we have to use the expected value of the next state.

To do this, we would multiply our three possible states by their probability. Now our expected value would be:

$0.8∗V(s′1)+0.1∗V(s′2)+0.1∗V(s3)$ <font size="1">**NB: the numbers are taken from the example for Non-Deterministic Search**</font>

Now the new Bellman equation for non-deterministic environments is:

$V_(s)=max_a(R(s,a)+γ∑s′P(s,a,s′)V(s′))$

This equation is what we'll be dealing with going forward since most realistic environments are stochastic in nature.

### 2.4 Q-Learning Intuition

Now that we understand the Bellman equation and understand a Markov decision process for the probability of the next state given an action, let's move on to Q-learning.

So far we've been dealing with the value of being in a given state, and we know we want to make an optimal decision about where to go next given our current state.

Now instead of looking at the value of each state $V(s)$, we're going to look at the value of each state-action pair, which is denoted by $Q(s,a)$.

Intuitively you can think of the Q-value as the quality of each action.Let's look at how we actually derive the value of $Q(s,a)$
by comparing is to $V(s)$.

As we just saw, here is the equation for $V(s)$

in a stochastic environment:

$$V_(s)=max_a(R(s,a)+γ∑s′P(s,a,s′)V(s′))$$

Here is the equation for $Q(s,a)$:

* By performing an action the first thing we get is a reward $R(s,a)$

* Now the agent is in the next state $s′$, and because the agent can end up in several states, we add the value of the next state which is the expected value of the next state 

$$Q(s,a)=R(s,a)+γ∑s′P(s,a,s′)V(s′))$$

What you will notice looking at this equation is that $Q(s,a)$ is the same value as what's in the brackets of the Bellman equation: $R(s,a)+γ∑s′P(s,a,s′)V(s′))$.

Here's how we can think about this intuitively:

The value of a state $V(s)$ is the maximum of all the possible Q-values.

Let's take this equation a step further by getting rid of $V$, since $V$ is a recursive function of $V$.

We're going to take the $V(s′)$ and replace it with $Q(s′,a′)$:

$$Q(s,a)=R(s,a)+γ∑s′P(s,a,s′)max_a′Q(s′,a′))$$

Now we have a recursive formula for the Q-value.

### 2.5 Temporal Difference

Temporal difference is an important concept at the heart of the Q-learning algorithm. This is how everything we've learned so far comes together in Q-learning. One thing we haven't mentioned yet about non-deterministic search is that it can be very difficult to actually calculate the value of each state.

Temporal difference is what allows us to calculate these values.

For now, we're just going to use the deterministic Bellman equation for simplicity, which to recap is: $Q(s,a)=R(s,a)+γmax_a′Q(s′,a′)$, but we'll use this refer to stochastic environments.

So we know that before an agent takes an action it has a Q-value: $Q(s,a)$.
After an action is taken we know what reward the agent actually got and what the value of the new state is: $R(s,a)+γmax_a′Q(s′,a′)$.

The temporal difference is defined as follows:

$TD(a,s)=R(s,a)+γmaxa′Q(s′,a′)−Qt−1(s,a)$

The first element is what we get after taking an action and the second element is the previous Q-value.

The question is are these two values the same?

Ideally, these two values should be the same since the first, $R(s,a)+γmax_a′Q(s′,a′)$ is the formula for calculating $Q(s,a)$.
But these values may not be the same because of the randomness that exists in the environment.

The reason it's called temporal difference is because of time. We have $Q(s,a)$ which is our previous Q-value and we have our new Q-value, which is $R(s,a)+γmaxa′Q(s′,a′)$.

So the question is: has there been a difference between these values in time?

Now that we have our temporal difference, here's how we use it:

$Qt(s,a)=Qt−1(s,a)+αTDt(a,s)$

where:

$α$ - is our learning rate.

So we take our previous $Qt−1(s,a)$ and add on the temporal difference times the learning rate to get our new $Qt(s,a)$.
This equation is how our Q-values are updated over time. Let's now plug in the $TD(a,s)$ equation into our new Q-learning equation:

$Qt(s,a)=Qt−1(s,a)+α(R(s,a)+γmax_a′Q(s′,a′)−Qt−1(s,a))$

We now have the full Q-learning equation, so let's move on to deep Q-learning.

### 2.6 Deep Q-Learning Intuition

In deep Q-learning we are, of course, making use of neural networks. In terms of the neural network we feed in the state, pass that through several hidden layers (the exact number depends on the architecture) and then output the Q-values. Here is a good visual representation of Q-learning vs. deep Q-learning from Analytics Vidhya:

<img src="./IMG/Q-learining_vs_Deep_Q-learning.webp" width="600">

You may be wondering why we need to introduce deep learning to the Q-learning equation. Q-learning works well when we have a relatively simple environment to solve, but when the number of states and actions we can take gets more complex we use deep learning as a function approximator. Let's look at how the equation changes with deep Q-learning.

Recall the equation for temporal difference:

$TD(a,s)=R(s,a)+γmax_a′Q(s′,a′)−Qt−1(s,a)$

In the maze example, the neural network will predict 4 values: up, right, left, or down.We then take these 4 values and compare it to the values that were previously predicted, which are stored in memory.

So we're comparing Q1 vs. Q−Target1, Q2 vs. Q−Target2, etc.
Recall that neural networks work by updating their weights, so we need to adapt our temporal difference equation to leverage this. So what we're going to do is calculate a loss by taking the sum of the squared differences of the Q-values and their targets:

$L=∑(Q−Target−Q)^2$.

We then take this loss and use backpropagation, or stochastic gradient descent, and pass it through the network and update the weights. This is the learning part, now let's move on to how the agent selects the best action to take.To choose which action is the best, we use the Q-values that we have and pass them through a softmax function. This process happens every time the agent is in a new state.

### 2.7 Experience Replay

Now that we've discussed how to apply neural networks to Q-learning, let's review another important concept in deep Q-learning: experience replay.
One thing that can lead to our agent misunderstanding the environment is consecutive interdependent states that are very similar.

For example, if we're teaching a self-driving car how to drive, and the first part of the road is just a straight line, the agent might not learn how to deal with any curves in the road.
This is where experience replay comes in.

From our self-driving car example, what happens with experience replay is that the initial experiences of driving in a straight line don't get put through the neural network right away.
Instead, these experiences are saved into memory by the agent.
Once the agent reaches a certain threshold then we tell the agent to learn from it.

So the agent is now learning from a batch of experiences. From these experiences, the agent randomly selects a uniformly distributed sample from this batch and learns from that.

Each experience is characterized by that state it was in, the action it took, the state it ended up in, and the reward it received.
By randomly sampling from the experiences, this breaks the bias that may have come from the sequential nature of a particular environment, for example driving in a straight line.

### 2.8 Action Selection Policies

Now that we understand how deep Q-learning uses experience replay to learn from a batch of experiences stored in memory, let's finish off with how the agent actually selects an action.

So the question is: once we have the Q-values, how do decide which one to use?
Recall that in simple Q-learning we just choose the action with the highest Q-value.

As we mentioned earlier, with deep Q-learning we pass the Q-values through a softmax function.
In reality, the action selection policy doesn't need to be softmax, there are others that could be used, and a few of the most common include:

* $ϵ-greedy$

* $ϵ-soft$

* Softmax

The reason that we don't just use the highest Q-value comes down to an important concept in reinforcement learning: the exploration vs. exploitation dilemma.

To summarize this concept, an agent must make a tradeoff between taking advantage of what it already knows about the environment, or exploring further.
If it takes advantage of what it already knows it could gain more rewards, but if it doesn't explore further it may not be choosing the optimal actions.

# 3. Implementation of Deep Q-Learning and how to match DeepMind’s score in Breakout

We are going to explore the method of implementing Deep Q-Networks (DQN) by following the outlined roadmap and explore it problems and resolved them.

python_version() ### 3.1. Reinfocement Learning in Deep Q-Learning

In supervised learning, a neural network learns to map inputs to outputs based on labeled training data. For instance, in classifying cats and dogs, the network is trained with labeled images until it can accurately classify new images.

In reinforcement learning, the approach is more akin to a child learning a new game. The computer explores the environment, receives occasional rewards when the score increases, and learns to make decisions by evaluating actions that lead to higher scores.

In contrast to supervised learning, reinforcement learning (RL) faces challenges with "sparse" and "delayed" rewards, as mentioned by [Mnih et al. 2013](https://arxiv.org/abs/1312.5602). "Sparse" implies infrequent rewards, and "delayed" refers to the significant time gap between an action and the corresponding reward. In RL scenarios, the agent might not receive feedback immediately after making a decision, making it challenging to associate actions with their eventual outcomes. For instance, in a maze, choosing a path at a fork may not yield immediate feedback; the reward, finding gold, comes later. To address this, RL uses a discount factor $\gamma$ to prioritize immediate rewards over delayed ones.

The concept introduced by Sutton and Barto in 2018 on page 55 is that of "discounting" in the context of reinforcement learning. In this approach, the agent aims to select actions to maximize the sum of the discounted rewards it receives over the future. Specifically, it chooses an action $A_t$ to maximize the "expected discounted return.

The discounted return $G_t$ is calculated as follows  

\begin{align}
G_{t} &= R_{t+1} + \gamma R_{t+2} + \gamma^2 R_{t+3} + ... \\
&=R_{t+1} + \gamma \bigl(R_{t+2} + \gamma R_{t+3} + ...\bigr)\\
&= R_{t+1} + \gamma G_{t+1}
\end{align}

where $R_t$ is the reward the agent received at time $t$.

$R_t$ is the reward received at time $t+i$,

$\gamma$ is the discount factor between 0 and 1, and andThe summation represents the accumulation of future rewards, each discounted by a factor of $\gamma$. 

This formulation allows the agent to prioritize more immediate rewards over those in the distant future, helping in decision-making processes where the consequences of actions unfold over time.

The authors furthermore explain:

>a reward received $k$ time steps in the future is worth only $\gamma^{k−1}$ times what it would be worth if it were received immediately.

and

>As $\gamma$ approaches 1, the return objective takes future rewards into account more strongly; the agent becomes more farsighted.

Let us look at a very simple example where there is just one reward not equal to 0 (calculate from right to left):

time $t$ |  0| 1 | 2|3|4|
:---| --- | ---| ---|---|---|
reward sequence $R_t$|  | 0 | 0 | 0 | 1 |
discounted returns $G_t$| $\gamma^3$ | $\gamma^2$ | $\gamma$ | 1 | 0 |
for $\gamma=0.9$|0.729 | 0.81 | 0.9| 1 |0|

Minor detail: In this example there is no reward at $t=0$ because the first reward $R_1$ is a result of action $a_0$ taken in state $s_0$ and the reward is always associated with the next state in Sutton and Barto. Some articles associate the reward $R_i$ with the state $s_i$ and action $a_i$ that caused the reward. In that case the definition of the discounted return changes to $G_t=R_{t} + \gamma G_{t+1}$, in the table the 1's would all be in the same column and there would be a reward equal to 0 at $t=0$ but not at $t=4$ .

Simply put, by discounting returns, future rewards increase past or current returns and the closer $\gamma$ is to 1, the farther the agent can *see* into the future. 

And *sparse*?
In our fictive maze example, the rewards are the sparser, the less gold you find. For an agent, a game is more difficult to learn, the sparser the reward is. "Pong" is one of the games DQN can learn fastest because the score changes quite often. "Montezuma's Revenge", on the other hand, has very sparse rewards and DQN (at least without some additional tricks) is not able to learn the game at all.

### 3.2 Q-learning in Deep Q-learning


So how does Q-learning work? If the agent (regardless if trained or still untrained) is shown a state $s$ of the game, it has to decide which action $a$ to perform (for example move paddle left or right in breakout). How does it do that? On page 2 [Mnih et al. 2013](https://arxiv.org/abs/1312.5602) define the so-called $Q$-Function:

>We define the optimal action-value function $Q^∗(s, a)$ as the maximum expected return achievable by following any strategy, after seeing some sequence $s$ and then taking some action $a$ 

This means that given a state of the game $s$ (for now please consider *sequences* as states of the game), $Q^*(s,a)$ is the best (discounted) total return the agent can achieve if it performs action $a$ in the current state $s$. So how does it choose which action to perform assuming we already know $Q^*(s,a)$? One obvious strategy would be to always choose the action with the maximum value of $Q^*$ (we will see later, why this is slightly problematic). But first of all, we need to find this magical function $Q^*$:

Let's say we are in state $s$, decide to perform action $a$, and arrive in the next state $s'$. If we assume that in state $s'$ the $Q^*$-values for all possible actions $a'$ were already known, then the $Q^*$-value in state $s$ for action $a$ (action-value in $s$ for action $a$) would be the reward $r$ we got for performing action $a$ plus the discounted maximum future return in $s'$:

\begin{equation}
Q^*(s,a) = r + \gamma \textrm{max}\left( Q^*(s',a') \right)
\end{equation}

Less formal but more intuitive:
If we were at a certain fork in the maze (state $s$) and want to know how good it was to choose the left path (perform action $a$), we add the gold we found after going left (reward we received for performing action $a$) and the amount of gold we expect to find down the road (maximum future discounted return). If you have to choose between finding a golden coin immediately after going left at a fork but nothing else down the road or finding nothing immediatley after going right but a treasure down the road, well, the right path is worth more/has a higher action-value.

This is the so-called **Bellman equation**. Deep Q-Learning uses a neural network to find an approximation $Q(s,a,\theta)$ of $Q^*(s,a)$. $\theta$ are the parameters of the neural network. We will discuss later, how exactly the parameters of the network are updated. Now, I will explain, how the neural network maps a state $s$ to $Q$-values for the possible actions $a$.

Earlier I mentioned, that I regard a *sequence* as a *state*. What did I mean with that? Imagine you have a pin-sharp image of a flying soccer ball. Can you tell in which direction it moves? No, you cannot, but you could if there was some kind of motion blur in the picture or if you had several images taken quickly one after another showing the ball in a slightly different position every time. The same problem occurs in Atari games. From a single frame of the game "Pong", the agent can not discern in which direction the ball moves. DeepMind met this problem by stacking several consecutive frames and considering this sequence a state that is passed to the neural network. From such a sequence the agent is able to detect the direction and speed of movement because the ball is in a different position in each frame.

>Since the agent only observes images of the current screen [...] it is impossible to fully understand the current situation from only the current screen $x_t$. We therefore consider sequences of actions and observations, $s_t = x_1, a_1, x_2, ..., a_{t−1}, x_t$, and learn game strategies that depend upon these sequences. All sequences in the emulator are assumed to terminate in a finite number of time-steps. This formalism gives rise to a large but finite Markov decision process (MDP) in which each sequence is a distinct state. ([page 2 of Mnih et al. 2013](https://arxiv.org/abs/1312.5602))


On page 5 of [Mnih et al. 2013](https://arxiv.org/abs/1312.5602) the authors explain the preprocessing of the frames:

>Working directly with raw Atari frames, which are 210 × 160 pixel images with a 128 color palette, can be computationally demanding, so we apply a basic preprocessing step aimed at reducing the input dimensionality. The raw frames are preprocessed by first converting their RGB representation to gray-scale and down-sampling it to a 110×84 image. The final input representation is obtained by cropping an 84 × 84 region of the image that roughly captures the playing area. The final cropping stage is only required because we use the GPU implementation of 2D convolutions from [...], which expects square inputs. For the experiments in this paper, the function $\phi$ [...] applies this preprocessing to the last 4 frames of a history and stacks them to produce the input to the $Q$-function.

So let us start by looking at how the preprocessing can be implemented. I used `gym` from OpenAi to provide the environment. A frame returned by the environment has the shape `(210,160,3)` where the 3 stands for the RGB color channels. Such a frame is passed to the Functor`FrameProcessor` which transforms it to a `(84,84,1)` frame, where the 1 indicates that instead of three RGB channels there is one grayscale channel. 


In [10]:
class FrameProcessor(object):
    """Resizes and converts RGB Atari frames to grayscale"""
    def __init__(self, frame_height=84, frame_width=84):
        """
        Args:
            frame_height: Integer, Height of a frame of an Atari game
            frame_width: Integer, Width of a frame of an Atari game
        """
        self.frame_height = frame_height
        self.frame_width = frame_width
        self.frame = tf.placeholder(shape=[210, 160, 3], dtype=tf.uint8)
        self.processed = tf.image.rgb_to_grayscale(self.frame)
        self.processed = tf.image.crop_to_bounding_box(self.processed, 34, 0, 160, 160)
        self.processed = tf.image.resize_images(self.processed, 
                                                [self.frame_height, self.frame_width], 
                                                method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    
    def __call__(self, session, frame):
        """
        Args:
            session: A Tensorflow session object
            frame: A (210, 160, 3) frame of an Atari game in RGB
        Returns:
            A processed (84, 84, 1) frame in grayscale
        """
        return session.run(self.processed, feed_dict={self.frame:frame})

## 3. Dueling Networks

Instead of the network architecture described in [Mnih et al. 2013](https://arxiv.org/abs/1312.5602) or [Mnih et al. 2015](https://www.nature.com/articles/nature14236/) I used the dueling network architecture described in [Wang et al. 2016](https://arxiv.org/abs/1511.06581).

<img src="./IMG/dueling.png" width="600">

Both the [Mnih et al. 2015](https://www.nature.com/articles/nature14236/) and the [Wang et al. 2016](https://arxiv.org/abs/1511.06581) dueling architecture have the same low-level convolutional structure:

>The first convolutional layer has 32 8x8 filters with stride 4, the second 64 4x4 filters with stride 2, and the third and final convolutional layer consists 64 3x3 filters with stride 1.

In the normal DQN architecture (top network in the figure) the *final hidden layer is fully-connected and consists of 512 rectifier units. The output layer is a fully-connected linear layer with a single output for each valid action.* (see page 6 of [Mnih et al. 2015](https://www.nature.com/articles/nature14236/)) 

These outputs are the predicted $Q(s,a;\theta)$-values for action $a$ in state $s$.

Instead of directly predicting a single $Q$-value for each action, the dueling architecture splits the final convolutional layer into two streams that represent the value and advantage functions that predict a *state value* $V(s)$ that depends only on the state, and *action advantages* $A(s,a)$ that depend on the state and the respective action. On page 2 of [Wang et al. 2016](https://arxiv.org/abs/1511.06581) the authors explain:

>Intuitively, the dueling architecture can learn which states are (or are not) valuable, without having to learn the effect of each action for each state. This is particularly useful in states where its actions do not affect the environment in any relevant way. 
In the experiments, we demonstrate that the dueling architecture can more quickly identify the correct action during policy evaluation as redundant or similar actions are added to the learning problem. 

The *state value* $V(s)$ predicts *how good it is to be in a certain state* $s$ and the *action advantage* $A(s,a)$ predicts *how good it is to perform action $a$ in state $s$*.
I suggest you take a look at figure 2 in [Wang et al. 2016](https://arxiv.org/abs/1511.06581) to better understand what the value- and advantage-stream learn to look at.

Next, we have to combine the value and advantage stream into $Q$-values $Q(s,a)$. This is done the following way (equation 9 in [Wang et al. 2016](https://arxiv.org/abs/1511.06581)):

\begin{equation}
Q(s,a) = V(s) + \left(A(s,a) - \frac 1{| \mathcal A |}\sum_{a'}A(s, a')\right)
\end{equation}

Why so complicated instead of just adding $V(s)$ and $A(s,a)$? Let's assume $Q(s,a) = V(s) + A(s,a)$:

 The Q function measures the value of choosing a particular action when in a particular state. The value function $V$, which is the expected value of $Q$ over all possible actions, $V = E(Q)$, measures how good it is to be in this particular state. 

If you combine $E(Q) = V$ and $Q = V + A$, you find $E(Q) = E(V) + E(A)$. But $V$ does not depend on any action, which means $E(V)=V$, $E(Q) = V + E(A) = V$ and thus $E(A)=0$. The expected value of the advantage $A(s,a')$ over all possible actions $a'$ has to be zero. The expected value of the expression in parentheses is zero because we subtract the mean of the advantages from every advantage:

\begin{equation}
E\left(A(s,a) - \frac 1{| \mathcal A |}\sum_{a'}A(s, a')\right) = E\Bigl(A - E(A)\Bigr) = E(A)- E(E(A)) = E(A) - E(A) = 0
\end{equation}

In the cell below you find the code that implements this architecture in tensorflow. Some things to keep in mind: You should normalize the input pixel values to [0,1] by dividing the input with 0xFF=255. The reason for this is, that the pixel values of the frames, the environment returns, are uint8 which can store values in the range [0,255]. 

**Furthermore, make sure you initialize the weights properly! 
The DQN uses the Relu activation function and the right initializer is [He et al. 2015 equation 10](https://arxiv.org/pdf/1502.01852v1.pdf) ([click here for a detailed explanation](https://www.youtube.com/watch?v=s2coXdufOzE&t=157s )).
In tensorflow use `tf.variance_scaling_initializer` [(documentation)](https://www.tensorflow.org/api_docs/python/tf/variance_scaling_initializer) with `scale = 2`.**

DeepMind used an implementation of the RMSProp optimizer that is different to the one in tensorflow (see page 23, eq 40 in [Graves 2014](https://arxiv.org/pdf/1308.0850v5.pdf)). Before implementing it myself, I tried the Adam optimizer which gave promising results without much hyperparameter-search. Adam was not invented when [Mnih et al. 2013](https://arxiv.org/abs/1312.5602) was published, so one could argue that they might have used it instead of RMSProp if it had been invented earlier. On the other hand, the authors of this [blog post](https://blog.paperspace.com/intro-to-optimization-momentum-rmsprop-adam/) compare *Momentum, RMSProp and Adam* and argue:
>Out of the above three, you may find momentum to be the most prevalent, despite Adam looking the most promising on paper. Empirical results have shown that all these algorithms can converge to different optimal local minima given the same loss. However, SGD with momentum seems to find more flatter minima than Adam, while adaptive methods tend to converge quickly towards sharper minima. Flatter minima generalize better than sharper ones.

Maybe some of these differences might be mitigated by the use of *AdamW*, which I described [in this blog post](https://medium.com/@fabiograetz/why-adamw-matters-736223f31b5d) (irrelevant for DQN since no regularization is used). For now, I stick with Adam and if I find some time in the future, I might come back to this since it might be well worth spending some time on playing with different optimizers and implementing the version of RMSProp used by DeepMind. 

*Edit*: In a later DeepMind paper called "Rainbow: Combining Improvements in Deep Reinforcement Learning" by [Hessel et al. 2017](https://arxiv.org/abs/1710.02298) RMSProp was substituted for Adam with a learning rate of 0.0000625 (see Table 1). This learning rate is close to what I found working well for Breakout (0.00001) before reading Hessel et al. 2017.

If you compare the dueling architecture described above to the network implemented in the next cell, you will find a small difference. Instead of two hidden fully connected layers with 512 rectifier units for each, the value and the advantage stream, I ended up adding a fourth convolutional layer with 1024 filters. The output has the shape (1, 1, 1024) and is then split into two streams with shapes (1, 1, 512). This architecture is suggested [here](https://github.com/awjuliani/DeepRL-Agents/blob/master/Double-Dueling-DQN.ipynb) and after performing some tests on the environment Pong, which is comparably easy to learn for a DQN agent, I find that this small adjustment lets the reward increase slightly earlier and higher.

# Refferences
[1] How to match DeepMind’s Deep Q-Learning score in Breakout by Fabio M. Graetz, Aug 26, 2018

[2] Deep Reinforcement Learning: Guide to Deep Q-Learning by Peter Foy